In [19]:
!pip install transformers --quiet
!pip install datasets --quiet
!pip install torch --quiet
#
!pip install openprompt
!nvidia-smi
!pip install datasets

Tue Feb 13 13:32:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [20]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import torch

In [ ]:
from datasets import load_dataset
from openprompt.data_utils import InputExample
from transformers import  AdamW, get_linear_schedule_with_warmup
from openprompt.prompts import MixedTemplate
from openprompt import PromptDataLoader
from openprompt import PromptForClassification
from openprompt.prompts import ManualVerbalizer
import torch
from pprint import pprint
from openprompt.plms import load_plm

In [3]:
#check the avaialbility of cuda
import torch
torch.cuda.is_available()


# 1. Data processing

In [14]:
# from google.colab import drive
# drive.mount('/content/drive')
# !ls '/content/drive/MyDrive/thesis/'

import json, csv
from datasets import Dataset, DatasetDict


### Pairs to dict


In [8]:
# df to dict
def df_to_dict(df):
    data_dicts = []
    for idx, row in df.iterrows():  # Use iterrows for DataFrame iteration
        concept1, concept2, label = row['concept1'], row['concept2'], row['label'] # Access columns correctly
        data_dicts.append({
            'concept1': concept1,
            'concept2': concept2,
            'idx': idx,
            'label': label
        })
    return data_dicts

In [9]:
import pandas as pd

# Read the data from the "data-base" folder
dev = pd.read_csv('./data/data-base/data_dev.csv')
test = pd.read_csv('./data/data-base/data_test.csv')
train = pd.read_csv('./data/data-base/data_train.csv')

dev_dict = df_to_dict(dev)
test_dict = df_to_dict(test)
train_dict = df_to_dict(train)

# 2. Training **

In [13]:
from datasets import load_dataset
from openprompt.data_utils import InputExample
from transformers import  AdamW, get_linear_schedule_with_warmup
from openprompt.prompts import MixedTemplate
from openprompt import PromptDataLoader
from openprompt import PromptForClassification
from openprompt.prompts import ManualVerbalizer
import torch
from pprint import pprint
from openprompt.plms import load_plm

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [15]:
data_dicts_split = {
    'train': train_dict,
    'validation': dev_dict,
    'test': test_dict
}

print(len(data_dicts_split['train']))
print(len(data_dicts_split['validation']))
print(len(data_dicts_split['test']))

2118
202
402


In [16]:
# data_dicts_split to dataset (read-to-use one)
dataset = {}
for split in ['train', 'validation', 'test']:
    dataset[split] = []
    for data in data_dicts_split[split]:
        input_example = InputExample(text_a = data['concept1'], text_b = data['concept2'], label=int(data['label']), guid=int(data['idx']))
        dataset[split].append(input_example)
print(dataset['train'][0])

{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "Concurrency (computer science)",
  "text_b": "Integral",
  "tgt_text": null
}



## Defining promtp model

In [35]:
# #construct a prompt model
# from openprompt import PromptForClassification
# promptModel = PromptForClassification(
#     template = promptTemplate,
#     plm = plm,
#     verbalizer = promptVerbalizer,
# )

# #load dataset to PromptDataloader
# from openprompt import PromptDataLoader
# data_loader = PromptDataLoader(
#     dataset = dataset,
#     tokenizer = tokenizer,
#     template = promptTemplate,
#     tokenizer_wrapper_class=WrapperClass,
# )

In [19]:
# load plm and tokenizer, and initialize template
plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")

mytemplate = MixedTemplate(model=plm, tokenizer=tokenizer, text='{"soft":"Is"} {"placeholder":"text_a"} {"soft":"a prerequisite of"}{"placeholder":"text_b"}?{"mask"}.')
wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
pprint(wrapped_example)
print(dataset['train'][0])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:217: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) beh

[[{'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 1, 'text': '▁I'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 2, 'text': 's'},
  {'loss_ids': 0,
   'shortenable_ids': 1,
   'soft_token_ids': 0,
   'text': ' Concurrency (computer science)'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 3, 'text': '▁'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 4, 'text': 'a'},
  {'loss_ids': 0,
   'shortenable_ids': 0,
   'soft_token_ids': 5,
   'text': '▁prerequisite'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 6, 'text': '▁of'},
  {'loss_ids': 0,
   'shortenable_ids': 1,
   'soft_token_ids': 0,
   'text': 'Integral'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 0, 'text': '?'},
  {'loss_ids': 1, 'shortenable_ids': 0, 'soft_token_ids': 0, 'text': '<mask>'},
  {'loss_ids': 0, 'shortenable_ids': 0, 'soft_token_ids': 0, 'text': '.'}],
 {'guid': 0, 'label': 0}]
{
  "guid": 0,
  "label": 0,
  "meta": {},
  "text_a": "Concurre

In [20]:
#todo: read params documentation
wrapped_t5tokenizer=WrapperClass(max_seq_length=128,decoder_max_length=3,tokenizer=tokenizer,truncate_method='head')
train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=8,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
validation_dataloader = PromptDataLoader(dataset=dataset["validation"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=8,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 2118it [00:09, 228.95it/s]
tokenizing: 202it [00:00, 253.85it/s]


In [21]:
#define verbalizer : mapping from logits on the vocabulary to the final label probability
classes = [
     1,
     0,
]
myverbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        1: ["yes", "correct", "true", "right", "correct", "true", "right"],
        0: ["no", "incorrect", "false", "wrong", "incorrect", "false", "wrong"],
    },
    tokenizer=tokenizer,
)
print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm
print('Vocab length:',len(tokenizer))
print('For each kind of possibility:')
myverbalizer.process_logits(logits)

Parameter containing:
tensor([[[ 4273],
         [ 2024],
         [ 1176],
         [  269],
         [ 2024],
         [ 1176],
         [  269]],

        [[  150],
         [12153],
         [ 6136],
         [ 1786],
         [12153],
         [ 6136],
         [ 1786]]])
Vocab length: 32100
For each kind of possibility:


tensor([[-3.3017, -2.6786],
        [-3.0837, -2.7418]])

In [ ]:
use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model = prompt_model.cuda()
#---

loss_func=torch.nn.CrossEntropyLoss()
no_decay=['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters1 = [
    {'params': [p for n, p in prompt_model.plm.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.plm.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# Using different optimizer for prompt parameters and model parameters
optimizer_grouped_parameters2 = [
    {'params': [p for n,p in prompt_model.template.named_parameters() if "raw_embedding" not in n]}
]

#---
for n,p in prompt_model.named_parameters():
    print(n)
optimizer1 = AdamW(optimizer_grouped_parameters1, lr=1e-4)
optimizer2 = AdamW(optimizer_grouped_parameters2, lr=1e-3)


In [ ]:
#----
from tqdm import tqdm
loss_list=[]
for epoch in range(1):
    tot_loss = 0
    for step, inputs in enumerate(tqdm(train_dataloader)):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer1.step()
        optimizer1.zero_grad()
        optimizer2.step()
        optimizer2.zero_grad()
        loss_list.append(loss.item())
    tot_loss=tot_loss/len(train_dataloader)
    print('Training Loss: {:.6f}'.format(tot_loss))
#---
import matplotlib.pyplot as plt
plt.figure(dpi=200)
plt.rcParams['font.family']='serif'
plt.plot(range(len(loss_list)),loss_list)

In [ ]:
allpreds = []
alllabels = []
for step, inputs in enumerate(tqdm(validation_dataloader)):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

In [ ]:
acc = sum([int(i==j) for i,j in zip(allpreds, alllabels)])/len(allpreds)
print(acc)